In [ ]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

#  Selenium driver 생성
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

BASE_URL = "https://ncpms.rda.go.kr/npms/NewIndcUserListR.np?indcMon=&indcSeq=&ncpms.cmm.token.html.TOKEN=ad588af4742072d7857a9b7092b7effa&pageIndex={}&sRegistDatetm=&eRegistDatetm=&sCrtpsnNm=&sIndcSj="

results = []

# 1~5페이지 반복
for page in range(1, 6):
    driver.get(BASE_URL.format(page))
    print(f"📄 현재 페이지: {page}")
    time.sleep(2)

    rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")

    for i, tr in enumerate(rows):
        try:
            a_tag = tr.find_element(By.CSS_SELECTOR, "td:nth-of-type(2) a[onclick]")
            onclick = a_tag.get_attribute("onclick")
            match = re.search(r"cfGoDtl\('(\d+)'\)", onclick)

            if match:
                indc_seq = match.group(1)
                title = a_tag.text.strip()

                a_tag.click()
                time.sleep(2)

                soup = BeautifulSoup(driver.page_source, "html.parser")

                # warning / watch / forecast 3개 모두 탐색
                for li_class in ["warning", "watch", "forecast"]:
                    li_block = soup.find("li", class_=li_class)
                    if li_block:
                        disease_tags = li_block.select("ul.afterClear li a")
                        for a in disease_tags:
                            disease_name = a.get_text(strip=True)
                            id_match = re.search(r"fncViewDtl\('([^']+)'\)", a['href'])
                            disease_id = id_match.group(1) if id_match else None

                            results.append({
                                "게시물 제목": title,
                                "indcSeq": indc_seq,
                                "li_class": li_class,
                                "병해 이름": disease_name,
                                "병해 ID": disease_id
                            })
                    else:
                        results.append({
                            "게시물 제목": title,
                            "indcSeq": indc_seq,
                            "li_class": li_class,
                            "병해 이름": None,
                            "병해 ID": None
                        })

                driver.back()
                time.sleep(2)

        except Exception as e:
            print(f"[{i+1}] 오류 발생: {e}")

driver.quit()

df = pd.DataFrame(results)
df.to_csv("pest_result.csv", index=False, encoding="utf-8-sig")

print("크롤링 완료!")
print(df.head())

In [ ]:
df.to_csv('disease_crawling.csv')